In [ ]:
import os
OPEN_AI_KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
from todos_service import tools ,tasks
tools

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
model=ChatOpenAI(model="gpt-4o-mini")
agent = create_tool_calling_agent(model, tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
tasks

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS


app = Flask(__name__)
CORS(app)

In [ ]:
def current_response(response):
    return response["messages"][len(response["messages"])-1].content

In [ ]:
@app.route('/agent', methods=['GET'])
def get_welcom():
    return "--------------welcom to agent ✨------------------"

@app.route('/agent', methods=['POST'])
def run_agent():
    """הפעלה של ה-Agent עם LangGraph"""

    user_prompt = request.json.get('prompt')
    response = agent_executor.invoke({"input": user_prompt})
    return {'response': response['output']} 

In [ ]:
if __name__ == '__main__':
   app.run(host='0.0.0.0', port=5000)